In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
#matplotlib figure size
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
from mri_project.custom_imports import *
from mri_project.utility import dfe
__spec__ = 'Mardas'

In [2]:
files = glob.glob("../data/images_with_predictions_v04/*data")
file = np.random.choice(files)

In [94]:
from uuid import uuid4
predicted = []
traced = []
def rescale(x):
    return np.uint8((x-x.min())/(x.max()-x.min())*255)
def map_fun(file, index=-1):
    print(index, file)
    if index % 40 == 0:
        print(f"{index}:{file}")
    data = joblib.load(file)
    data.get_traced_contours(89.9)
    data.get_predicted_contours(89.9)
    data.get_contour_areas()
    data.get_contour_centers() 
    dic = {
        'raw': data.raw_image,
        'traced': data.traced_image,
        'predicted': data.predicted,
        'traced_lever_arms_90': data.traced_lever_arm_images[89.9]+150,
        'predicted_lever_arms_90': data.predicted_lever_arm_images[89.9]+150,
    }
    d, f, e = dfe(data.id)
    dr = d.replace('mri_analysis', 'mri_analysis_behnam')\
        .replace(" Set#", '/Set').replace(' ', '__')\
        .replace('..', 'blahblah').replace('.', '_').replace('blahblah', '..')
    if not os.path.exists(dr):
        os.makedirs(dr)
    for k, v in dic.items():
        # plt.imshow(v)
        # plt.axis('off')
        # plt.savefig(f"{dr}/{k}.png", dpi=150, bbox_inches=0)
        f_name = f"{dr}/{k}_{f}.png"
        cv2.imwrite(f_name, rescale(v))
    return data.id, {'traced': data.traced_features, 'predicted': data.predicted_features}

In [95]:
# for index, file in enumerate(files):
#     if index % 40 == 0:
#         print(index, file)
#     print(mardas(file))
#     break

In [96]:
import dask.bag as db

In [97]:
res = db.from_sequence(files).map(map_fun).compute()

In [98]:
from flatten_dict import flatten

In [99]:
res_fl = flatten(dict(res))

In [100]:
df = pd.DataFrame(res_fl)
df = df.transpose().reset_index()
df['value'] = df[0].combine(df[1], lambda x, y: x if x == y else (x, y))
df = df.drop([0, 1], axis=1)
df.columns = ['id', 'type', 'metric', 'muscle', 'value']

In [107]:
df

,id,type,metric,muscle,value
0,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,1,0.00578261
1,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,4,4.13338
2,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,5,2.20564
3,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,6,2.36826
4,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,7,2.38706
...,...,...,...,...,...
50491,../data/mri_analysis/2.Multigroup Set#2/Weight...,predicted,center,7,"(584.3502538071066, 911.9873096446701)"
50492,../data/mri_analysis/2.Multigroup Set#2/Weight...,predicted,center,8,"(819.8612565445026, 702.6282722513089)"
50493,../data/mri_analysis/2.Multigroup Set#2/Weight...,predicted,center,9,"(580.306976744186, 711.6255813953488)"
50494,../data/mri_analysis/2.Multigroup Set#2/Weight...,predicted,center,10,"(1054.5827027027026, 600.5621621621622)"


In [112]:
df.pivot(index=['id', 'muscle'], columns=['type', 'metric'])['value']

type                                                              traced  \
metric                                                    lever_arm_89.9   
id                                                 muscle                  
../data/mri_analysis/1.Multigroup Set#1_DONE/Co... 1           0.0185627   
                                                   2             3.14821   
                                                   3             3.33941   
                                                   4             5.45915   
                                                   5             4.06823   
...                                                                  ...   
../data/mri_analysis/4.Contiguous Set#2/Weight ... 7             4.40486   
                                                   8             3.17714   
                                                   9             3.14522   
                                                   10            10.2864   
                                                   11            11.7649   

type                                                                \
metric                                                        area   
id                                                 muscle            
../data/mri_analysis/1.Multigroup Set#1_DONE/Co... 1        12.317   
                                                   2       5.47881   
                                                   3       5.70134   
                                                   4       2.12653   
                                                   5       26.8763   
...                                                            ...   
../data/mri_analysis/4.Contiguous Set#2/Weight ... 7       18.3015   
                                                   8       5.50711   
                                                   9       4.50121   
                                                   10      17.4392   
                                                   11      16.6216   

type                                                                                                \
metric                                                                                      center   
id                                                 muscle                                            
../data/mri_analysis/1.Multigroup Set#1_DONE/Co... 1        (704.6863753213368, 701.8071979434447)   
                                                   2        (587.8949579831933, 737.7142857142857)   
                                                   3        (827.2916666666666, 739.4541666666667)   
                                                   4       (502.75132275132273, 812.2169312169312)   
                                                   5               (554.2135416666666, 890.984375)   
...                                                                                            ...   
../data/mri_analysis/4.Contiguous Set#2/Weight ... 7       (769.8281505728314, 1025.4926350245498)   
                                                   8       (1162.9712460063897, 817.3450479233227)   
                                                   9         (834.8545034642033, 825.635103926097)   
                                                   10      (1531.6973451327433, 675.7619469026548)   
                                                   11       (387.2413101604278, 681.4110962566845)   

type                                                           predicted  \
metric                                                    lever_arm_89.9   
id                                                 muscle                  
../data/mri_analysis/1.Multigroup Set#1_DONE/Co... 1          0.00979603   
                                                   2             6.26093   
                                                   3             6.65649   
                                                   4             4.35

In [104]:
df.to_csv("../data/Fehmi/all_features_19996.csv", index=False)

In [93]:
df

level_0,id,muscle_number,area_predicted,center_predicted,lever_arm_89.9_predicted,area_traced,center_traced,lever_arm_89.9_traced
0,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,0,16.1074,"[716.3617021276596, 710.1306990881459]",0.00979603,12.317,"[704.6863753213368, 701.8071979434447]",0.0185627
1,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,1,7.20287,"[947.2331606217616, 827.1813471502591]",6.26093,5.47881,"[587.8949579831933, 737.7142857142857]",3.14821
2,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,2,8.64308,"[470.5659340659341, 819.3516483516484]",6.65649,5.70134,"[827.2916666666666, 739.4541666666667]",3.33941
3,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,3,14.2926,"[876.2478386167147, 448.64265129682997]",4.35512,2.12653,"[502.75132275132273, 812.2169312169312]",5.45915
4,../data/mri_analysis/1.Multigroup Set#1_DONE/C...,4,12.1503,"[540.6023622047244, 446.9488188976378]",4.74087,26.8763,"[554.2135416666666, 890.984375]",4.06823
...,...,...,...,...,...,...,...,...
8363,../data/mri_analysis/4.Contiguous Set#2/Weight...,6,23.2587,"[813.6892655367232, 1019.2730696798493]",3.36616,18.3015,"[769.8281505728314, 1025.4926350245498]",4.40486
8364,../data/mri_analysis/4.Contiguous Set#2/Weight...,7,9.0568,"[1159.73440285205, 803.3351158645277]",3.30863,5.50711,"[1162.9712460063897, 817.3450479233227]",3.17714
8365,../data/mri_analysis/4.Contiguous Set#2/Weight...,8,8.77558,"[840.8084714548803, 802.4475138121547]",2.83634,4.50121,"[834.8545034642033, 825.635103926097]",3.14522
8366,../data/mri_analysis/4.Contiguous Set#2/Weight...,9,31.1198,"[1515.9841849148418, 682.7232360097323]",10.1768,17.4392,"[1531.6973451327433, 675.7619469026548]",10.2864


In [10]:
df = pd.read_csv("../data/prediction_stats_v01.csv")

In [13]:
df.metric.unique()

array(['area_ratio', 'center_distance', 'lever_arm_89.8_distance',
       'total_accuracy', 'accuracy', 'precision', 'recall'], dtype=object)

In [17]:
df

,Unnamed: 0,metric,id,muscle,value,n_muscles
0,0,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,1.575099,9
1,1,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,3.791567,9
2,2,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,2.927293,9
3,3,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,1.532780,9
4,4,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,11,1.511721,9
...,...,...,...,...,...,...
46871,30515,recall,../data/mri_analysis/4.Contiguous Set#2/Weight...,7,0.975620,11
46872,30516,recall,../data/mri_analysis/4.Contiguous Set#2/Weight...,8,0.954185,11
46873,30517,recall,../data/mri_analysis/4.Contiguous Set#2/Weight...,9,0.913792,11
46874,30518,recall,../data/mri_analysis/4.Contiguous Set#2/Weight...,10,0.998315,11
